# Data Collection

### Importing Libraries

In [0]:
!pip install praw
!pip install nltk
import json
import pandas as pd
import requests
import datetime as dt
import praw
from datetime import timedelta 
import nltk 
from collections import defaultdict 
from nltk.stem import SnowballStemmer
snowball = SnowballStemmer(language = "english")
from nltk.corpus import stopwords
from nltk.corpus import words
import string
nltk.download('words')
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))
import regex as re

You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Get the list of tickers

In [0]:
import pandas as pd

tickers = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/ferasdahrooge@gmail.com/listoftickers-2.csv")
tickers = tickers.select("Symbol")
tickers= tickers.rdd.map(lambda x: x.Symbol).collect()
tickers.remove("OC")
tickers.remove("DD")
tickers.remove("TA")

# Remove single character tickers
tickers = [str(x) for x in tickers if len(str(x)) != 1]

tickers.append("SPY")
tickers.append("V") #Visa
tickers.append("T") #AT&T
tickers.append("C") #Citigroup

## CollectData From Pushshift API (DataCollection Part 1)

### Coded by: Feras and Ardit

In [0]:
d = {}
end_time = dt.datetime(2022, 11, 15)
start_time = dt.datetime(2022,11, 14)
url = 'https://api.pushshift.io/reddit/search/submission'
def collectData(subreddit, size):
    after = int(start_time.timestamp())
    before = int(end_time.timestamp())
    for i in range(1, 100):
        if i==1:
            params = {"subreddit":subreddit, 'size':1000, 'before': before, 'after': after}
        else:
            params = {"subreddit":subreddit, 'size':size,'before': before, 'after': after}
        res = requests.get(url, params)    
        data = res.json()
        dat = data['data']
        before = dat[-1]['created_utc']
        df_name = subreddit+str(i)
        d[df_name] = pd.DataFrame(dat)
        return d

### Build DataFrame

In [0]:
def buildDataFrame():
    dataFrame = collectData('wallstreetbets', 1000)
    df = dataFrame["wallstreetbets1"]
    df['Time of Creation'] = df['created_utc'].map(lambda t: dt.datetime.fromtimestamp(t))
    df = df[['id', 'title', 'selftext', 'author', 'score', 'num_comments','Time of Creation']]
    return df

### Save into Pickle File

In [0]:

while (end_time <= dt.datetime(2022, 11,15)):
    data = buildDataFrame()
    data = spark.createDataFrame(data) 
    data.rdd.saveAsPickleFile("FileStore/output/old score by day/{}.pkl".format(str(start_time.year) + '-' + str(start_time.month) + '-' + str(start_time.day)))
    start_time += dt.timedelta(days = 1)
    end_time += dt.timedelta(days = 1)


---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
<command-2038261933220532> in <module>
      2     data = buildDataFrame()
      3     data = spark.createDataFrame(data)
----> 4     data.rdd.saveAsPickleFile("FileStore/output/old score by day/{}.pkl".format(str(start_time.year) + '-' + str(start_time.month) + '-' + str(start_time.day)))
      5     start_time += dt.timedelta(days = 1)
      6     end_time += dt.timedelta(days = 1)

/databricks/spark/python/pyspark/rdd.py in saveAsPickleFile(self, path, batchSize)
   1800             ser = BatchedSerializer(PickleSerializer(), batchSize)
   1801         reserialized = self._reserialize(ser)
-> 1802         self.ctx._jvm.PythonRDD.saveAsPickleFile(reserialized._jrdd, path)
   1803 
   1804     def saveAsTextFile(self, path, compressionCodecClass=None):

/databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/java_gateway.py in __call__

### example of data output for DataCollection Part 1

In [0]:
data.show()

+------+--------------------+---------+------------------+-----+------------+-------------------+
|    id|               title| selftext|            author|score|num_comments|   Time of Creation|
+------+--------------------+---------+------------------+-----+------------+-------------------+
|yvgxje|The only way to e...|         |     WebbsPowerade|    1|           1|2022-11-14 23:59:22|
|yvgw60|I choose to belie...|         | mercilessfatehate|    1|           0|2022-11-14 23:57:35|
|yvgvph|Jim Cramer is a C...|         |        Ciasecrets|    1|           0|2022-11-14 23:57:01|
|yvgopq|An inverted yield...|         |        56Kdial_up|    1|           2|2022-11-14 23:48:31|
|yvghmj|Can you bring bac...|[removed]|        WisedKanny|    1|           0|2022-11-14 23:39:59|
|yvghk1|Definitely pulled...|         |  Nearby-Soil-5668|    1|           2|2022-11-14 23:39:52|
|yvg65f|Sell before earni...|[removed]|        FrankieH0T|    1|           1|2022-11-14 23:26:05|
|yvg5w3|         Day

## Fix Score Using Python Reddit API Wrapper (DataCollection Part 2)

### Use Praw to Extract the Right Score and Remove Bot Posts with scores over 100

In [0]:
def reddit_score_fix(df, client_id, client_secret, user_agent, password, username):
    df = df.toPandas()
    reddit = praw.Reddit(
        client_id=client_id,
        client_secret=client_secret,
        password=password,
        user_agent=user_agent,
        username=username,
    )
    min_score = 100
    df['new_id'] = "t3_" + df["id"]
    gentest = reddit.info(fullnames = df['new_id'].to_list())
    midlist = [[item.id, item.score] for item in gentest]
    df_midlist = pd.DataFrame(midlist, columns = ['ID', 'SCORE'])
    df_merged = pd.merge(df, df_midlist, left_on = 'id', right_on = 'ID', how = 'outer')
    df['score'] = df_merged['SCORE']

    df = df[df['score'] > min_score]
    return df

### Save into Pickle File

In [0]:
time = dt.datetime(2022, 11, 15)

while (time <= dt.datetime(2022, 11, 15)):
    try:
        pickleRdd = sc.pickleFile("FileStore/output/old score by day/{}.pkl".format(str(time.year) + '-' + str(time.month) + '-' + str(time.day))).collect()
        preFixed_score_data = spark.createDataFrame(pickleRdd)
    except FileNotFoundError:
        time += dt.timedelta(days=1)
        continue
    fixed_score_data = reddit_score_fix(preFixed_score_data, 'inJ6zJcKI7dSDOEnYtZtSg','HMVgC-yLRwpqsWAnYfPulFCna2ZbRA','ENSF612 - Pulling Score','mengsoftware','ensf612')
    fixed_score_data = spark.createDataFrame(fixed_score_data) 
    fixed_score_data.rdd.saveAsPickleFile("FileStore/output/new score by day/{}.pkl".format(str(time.year) + '-' + str(time.month) + '-' + str(time.day)))
    
    time += dt.timedelta(days=1)

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
<command-2038261933220537> in <module>
     10     fixed_score_data = reddit_score_fix(preFixed_score_data, 'inJ6zJcKI7dSDOEnYtZtSg','HMVgC-yLRwpqsWAnYfPulFCna2ZbRA','ENSF612 - Pulling Score','mengsoftware','ensf612')
     11     fixed_score_data = spark.createDataFrame(fixed_score_data)
---> 12     fixed_score_data.rdd.saveAsPickleFile("FileStore/output/new score by day/{}.pkl".format(str(time.year) + '-' + str(time.month) + '-' + str(time.day)))
     13 
     14     time += dt.timedelta(days=1)

/databricks/spark/python/pyspark/rdd.py in saveAsPickleFile(self, path, batchSize)
   1800             ser = BatchedSerializer(PickleSerializer(), batchSize)
   1801         reserialized = self._reserialize(ser)
-> 1802         self.ctx._jvm.PythonRDD.saveAsPickleFile(reserialized._jrdd, path)
   1803 
   1804     def saveAsTextFile(self, path

### example of data output for DataCollection Part 2

In [0]:
fixed_score_data.show()

+------+--------------------+--------------------+--------------------+-----+------------+-------------------+---------+
|    id|               title|            selftext|              author|score|num_comments|   Time of Creation|   new_id|
+------+--------------------+--------------------+--------------------+-----+------------+-------------------+---------+
|ywdnhw|bears: “AMZN will...|                    |  usayhesjustafriend|  189|           2|2022-11-15 23:51:53|t3_ywdnhw|
|ywd13d|GEO Group Ape Tho...|                    |Financial-Process-86|  304|           0|2022-11-15 23:25:43|t3_ywd13d|
|ywcxtu|MEME STOCK YOLO —...|                    |         Key-Ear7468|  288|           2|2022-11-15 23:21:53|t3_ywcxtu|
|ywb5p1|     Welp this is it|\-30k in June and...|         Tom-W-Davis| 2508|           1|2022-11-15 22:10:06|t3_ywb5p1|
|ywaooz|Are we gonna see ...|                    |Which-Instruction-67|  332|           2|2022-11-15 21:50:36|t3_ywaooz|
|ywaemz|Are you people fu...|   

## Including Ticker (DataCollection Part 3)

### Preprocess the DataCollected

In [0]:
# Based on Dr. Uddin's A1 Solution, stemming has been added

regex = re.compile('[^a-zA-Z]')

# Use regex to remove non-alpha characters
def removeNonAlpha(word):
    return regex.sub('', word)
 
def remove_specialchars(word):
    if word is None or word == "":
        return word
    exclude = set(string.punctuation)    
    exclude.add('..')
    exclude.add('*')
    for c in exclude:
        word = word.strip(c)
    return word

# Use snowball stemmer to stem the words
def stem(word):
    return snowball.stem(word)

def default_zero():
    return 0

# pass the sentences through this function to preprocess the text
def preprocess_text(text_input):
    text_input = str(text_input)
  
    words = []
    ticker_dict = defaultdict(default_zero)
    
    # Tokenize into sentences, then into words
    sents = nltk.sent_tokenize(text_input)
    for sent in sents:
        for word in nltk.word_tokenize(sent):
            
            # Remove stop words, specials chars, non-alpha, and stem the words
            if word.lower() in stop_words: continue
            word = remove_specialchars(word)

            # Check if any of the words corresponds to a stock ticker
            # Case sensitive at this point (assuming tickers are all caps)
            # Make the words all lowercase (unless ticker)
            if word in tickers:
                ticker_dict[word] += 1
            else:
                word = word.lower()
  
            word = removeNonAlpha(word)
#             word = stem(word)
            
            # Remove words with length shorter than 3 characters if it's not a ticker
            if len(word) < 3 and word not in ticker_dict.keys(): continue
            
            if word is not None:
                words.append(word)
                
        
    # Return the preprocessed text
    return " ".join(words), ticker_dict

### Include the rows that only contains tickers

In [0]:
def dataFrame_including_ticker(df):
    df['fulltext'] = df['title'] + " " + df['selftext']
    # Preprocess the title and body and determine if either contains a stock ticker
    df['fulltext_processed'], df['ticker'] = zip(*df['fulltext'].apply(preprocess_text))
    df.dropna(subset=['fulltext'])
    # Remove posts that don't contain a ticker in the title or body
    df = df[df['ticker'] != {}] 
    list_max = []
    ticker_dictionary_list = df['ticker'].tolist()
    for i in ticker_dictionary_list:
        list_max.append(max(i, key= i.get))
        
    df['ticker'] = list_max                
    return df

### Save them into a Pickle File

In [0]:
time = dt.datetime(2022,11,14)

while (time <= dt.datetime(2022, 11, 14)):
    try:
        pickleRdd = sc.pickleFile("FileStore/output/new score by day/{}.pkl".format(str(time.year) + '-' + str(time.month) + '-' + str(time.day))).collect()
        fixed_score_data = spark.createDataFrame(pickleRdd)
    except FileNotFoundError:
        time += dt.timedelta(days=1)
        continue
        
    fixed_score_data = fixed_score_data.toPandas()    
    if fixed_score_data.empty:
        time += dt.timedelta(days=1)
        continue
        
    fixed_has_ticker_dataFrame = dataFrame_including_ticker(fixed_score_data)
    fixed_has_ticker_dataFrame = spark.createDataFrame(fixed_has_ticker_dataFrame) 
    fixed_has_ticker_dataFrame.rdd.saveAsPickleFile("FileStore/output/main ticker/{}.pkl".format(str(time.year) + '-' + str(time.month) + '-' + str(time.day)))
    time += dt.timedelta(days=1)

<command-2038261933220543>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ticker'] = list_max


In [0]:
fixed_has_ticker_dataFrame.show()

+------+--------------------+--------------------+--------------------+-----+------------+-------------------+---------+--------------------+--------------------+------+
|    id|               title|            selftext|              author|score|num_comments|   Time of Creation|   new_id|            fulltext|  fulltext_processed|ticker|
+------+--------------------+--------------------+--------------------+-----+------------+-------------------+---------+--------------------+--------------------+------+
|yvdlow|Now short on~ max...|Just wanted to sa...|MikasalsTheBestWaifu|  106|           2|2022-11-14 21:43:47|t3_yvdlow|Now short on~ max...|short max margin ...|  ASTS|
|yvcgp5|What Are Your Mov...|**Watch WallStree...|OPINION_IS_UNPOPULAR|  165|           3|2022-11-14 21:00:11|t3_yvcgp5|What Are Your Mov...|moves tomorrow no...|    DM|
|yv839x|Amazon reportedly...|Amazon is plannin...|       predictany007|  105|           1|2022-11-14 18:27:08|t3_yv839x|Amazon reportedly...|amazon re

## Collect the data into one CSV File

In [0]:
time = dt.datetime(2022, 11, 14)
full_dataset = pd.DataFrame()

# Consolidate our data into one big dataframe
while (time <= dt.datetime(2022, 11, 14)):
    try:
        pickleRdd = sc.pickleFile("FileStore/output/main ticker/{}.pkl".format(str(time.year) + '-' + str(time.month) + '-' + str(time.day))).collect()
        main_ticker_dataFrame = spark.createDataFrame(pickleRdd)
    except FileNotFoundError:
        time += dt.timedelta(days=1)
        continue
    main_ticker_dataFrame = main_ticker_dataFrame.toPandas()
    full_dataset = pd.concat([full_dataset, main_ticker_dataFrame], ignore_index=True)
    
    time += dt.timedelta(days=1)

full_dataset = spark.createDataFrame(full_dataset)  
full_dataset.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").option("delimiter", "\t").option("compression", "gzip").save("dbfs:/FileStore/df/full_dataset.csv")
full_dataset.rdd.saveAsPickleFile("FileStore/output/full_dataset.pkl")